##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-2.0-flash-exp' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
# Used to securely store your API key
from google.colab import userdata

In [6]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [10]:
# Upload the video and print a confirmation.
video_file_name = "/content/drive/MyDrive/法学课程/刑事执行法1-1-1_x264.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/67v61znu1p6e


In [11]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)


.................

In [13]:
# 角色设定和任务描述
prompt = """角色设定: 你是一个学生，正在学习刑事执行，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。
任务描述: 请观看我下面给出的刑事执行法的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。
**问题列表**:
1.	 管制与拘役的主要区别是什么？
2.	 拘役的刑期一般是多长？
3.	 简述易科制度的概念。
4.	 请概括总结我国目前对罪犯的计分考核的主要方法。
5.	有期徒刑的刑期一般在什么范围？
6.	《刑法修正案八》对死缓的减刑政策做了什么调整？
7.	简述短期自由刑的主要弊端。
8.	分析为何德国刑法典强调“判处短期自由刑属于例外情况”？
9.	比较美国和欧洲在终身监禁（包括不得假释的终身监禁）的适用上的主要差异。
10.	 某人因危险驾驶（醉驾）被判处拘役3个月，同时被判处罚金。如果他无力缴纳罚金，根据易科制度的理念，可能会如何处理？请结合文本内容分析。
11.	 一位被判处有期徒刑8年的罪犯，服刑5年后表现良好，根据广东省监狱罪犯分级管理实施办法，他可能处于什么等级？如果该罪犯曾经被警告一次，会如何影响他
12.	如果你是一个立法者，你会如何看待我国目前拘役适用率迅速增长的现象？结合文本内容，你认为应该如何完善拘役制度？
13.	监狱对罪犯的计分考核制度，在鼓励罪犯改造的同时，可能会有哪些负面影响？结合文本信息进行推理。
14.	如何理解 “在注重提高短期自由刑执行质量的同时，尽量辅之以缓刑的适用”的观点？
15.	假设某地发生了一起重大经济犯罪案件，犯罪分子被判处无期徒刑，但在服刑期间表现良好，有重大立功表现，根据课程内容，你认为该犯罪分子能否减刑？请说明理由。
16.	如果一个国家的刑罚体系中完全废除了短期自由刑，你认为会对该国的刑事司法产生哪些影响？
17.	某地发生了一起严重的环境污染案件，犯罪分子被判处拘役，但其在服刑期间表现极差，屡次违反监狱规定，根据课程内容，你认为该犯罪分子的刑罚执行方式是否需要调整？请说明理由。
18.	假设某地发生了一起严重的暴力犯罪案件，犯罪分子被判处无期徒刑，但在服刑期间表现良好，有重大立功表现，根据课程内容，你认为该犯罪分子能否假释？请说明理由。
19.	如果一个国家的刑罚体系中，罚金刑的适用范围不断扩大，甚至可以适用于一些原本应判处短期自由刑的犯罪行为，你认为会对该国的刑事司法产生哪些影响？
20.	如果一个国家的刑罚体系中，社区矫正制度不断完善和发展，你认为会对该国的刑事司法产生哪些影响？
21.	1979年公安部《关于拘役的具体执行办法的通知》中，对于不具备建立拘役所条件的，罪犯可以被安排在哪里执行拘役？
22.	李斯特认为“短期自由刑的弊端使得旧的监牢变成了不道德行为的滋生地，变成了犯罪的高等学校”，这句话中“旧的监牢”具体指什么？
23.	课程中提到“霍华德认为，独居制不仅比肉刑更人道，而且也更有效”，霍华德提出独居制的理由是什么？
24.	根据《德国刑法典》第43条，若无法缴纳罚金，替代的自由刑的换算比例是什么？
25.	根据《监狱法》第15条，被判处有期徒刑的罪犯，若在交付执行刑罚前，剩余刑期在多长时间以下的，由看守所代为执行？
26.	联系宪法中关于基本制度和基本人民权利义务的规定，论述宪法是如何保障中国式现代化的？
27.	联系习近平法治思想，论述依法治国和依宪治国。
基于刚才的视频，请你开始针对问题列表的每个问题进行作答。对于无法推出答案的问题，请回答我暂时从视频中找不到答案。
"""

# 选择 Gemini 模型
model = genai.GenerativeModel(model_name="gemini-exp-1206")
chat = model.start_chat(history=[])

from tenacity import retry, stop_after_attempt

# 进行 LLM 请求
print("Making LLM inference request...")

@retry(stop=stop_after_attempt(5))  # <-- 关键部分
def generate_response(video_file, prompt):
    return chat.send_message([video_file, prompt], request_options={"timeout": 600})

# 调用生成响应的函数
response = generate_response(video_file, prompt)

# 打印响应，渲染任何 Markdown
print(response.text)

Making LLM inference request...


ERROR:tornado.access:500 POST /v1beta/models/gemini-exp-1206:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 99960.63ms


KeyboardInterrupt: 

In [11]:
chat.history

[parts {
   file_data {
     mime_type: "video/mp4"
     file_uri: "https://generativelanguage.googleapis.com/v1beta/files/ttg7e4976w7t"
   }
 }
 parts {
   text: "角色设定: 你是一个学生，正在学习刑事执行，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。\n任务描述: 请观看我下面给出的刑事执行法的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。\n由于视频较长，我会分为上下部分上传，上面上传的视频为前半部分。\n"
 }
 role: "user",
 parts {
   text: "好的，我已经准备好观看您提供的刑事执行法课程视频的上半部分。请上传视频，我会仔细观看，然后尽力清晰、准确地回答您的问题，并结合视频中的相关信息来支持我的答案。让我们开始吧！\n由于你还没有上传视频，我现在无法提供具体的分析和回答。请上传视频后，我将立即开始学习。\n另外，针对视频的下半部分，我也已做好准备，请在合适的时候上传，谢谢！\n"
 }
 role: "model"]

In [12]:
# Upload the video and print a confirmation.
video_file_name = "/content/drive/MyDrive/法学课程/刑事执行法1-2-1_x264.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)


Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/skhsfmzw912c
........

In [17]:
prompt = """我已经给你上传了后半部分视频。**问题列表**:
1.	 管制与拘役的主要区别是什么？
2.	 拘役的刑期一般是多长？
3.	 简述易科制度的概念。
4.	 请概括总结我国目前对罪犯的计分考核的主要方法。
5.	有期徒刑的刑期一般在什么范围？
6.	《刑法修正案八》对死缓的减刑政策做了什么调整？
7.	简述短期自由刑的主要弊端。
8.	分析为何德国刑法典强调“判处短期自由刑属于例外情况”？
9.	比较美国和欧洲在终身监禁（包括不得假释的终身监禁）的适用上的主要差异。
10.	 某人因危险驾驶（醉驾）被判处拘役3个月，同时被判处罚金。如果他无力缴纳罚金，根据易科制度的理念，可能会如何处理？请结合文本内容分析。
11.	 一位被判处有期徒刑8年的罪犯，服刑5年后表现良好，根据广东省监狱罪犯分级管理实施办法，他可能处于什么等级？如果该罪犯曾经被警告一次，会如何影响他
12.	如果你是一个立法者，你会如何看待我国目前拘役适用率迅速增长的现象？结合文本内容，你认为应该如何完善拘役制度？
13.	监狱对罪犯的计分考核制度，在鼓励罪犯改造的同时，可能会有哪些负面影响？结合文本信息进行推理。
14.	如何理解 “在注重提高短期自由刑执行质量的同时，尽量辅之以缓刑的适用”的观点？
15.	假设某地发生了一起重大经济犯罪案件，犯罪分子被判处无期徒刑，但在服刑期间表现良好，有重大立功表现，根据课程内容，你认为该犯罪分子能否减刑？请说明理由。
16.	如果一个国家的刑罚体系中完全废除了短期自由刑，你认为会对该国的刑事司法产生哪些影响？
17.	某地发生了一起严重的环境污染案件，犯罪分子被判处拘役，但其在服刑期间表现极差，屡次违反监狱规定，根据课程内容，你认为该犯罪分子的刑罚执行方式是否需要调整？请说明理由。
18.	假设某地发生了一起严重的暴力犯罪案件，犯罪分子被判处无期徒刑，但在服刑期间表现良好，有重大立功表现，根据课程内容，你认为该犯罪分子能否假释？请说明理由。
19.	如果一个国家的刑罚体系中，罚金刑的适用范围不断扩大，甚至可以适用于一些原本应判处短期自由刑的犯罪行为，你认为会对该国的刑事司法产生哪些影响？
20.	如果一个国家的刑罚体系中，社区矫正制度不断完善和发展，你认为会对该国的刑事司法产生哪些影响？
21.	1979年公安部《关于拘役的具体执行办法的通知》中，对于不具备建立拘役所条件的，罪犯可以被安排在哪里执行拘役？
22.	李斯特认为“短期自由刑的弊端使得旧的监牢变成了不道德行为的滋生地，变成了犯罪的高等学校”，这句话中“旧的监牢”具体指什么？
23.	课程中提到“霍华德认为，独居制不仅比肉刑更人道，而且也更有效”，霍华德提出独居制的理由是什么？
24.	根据《德国刑法典》第43条，若无法缴纳罚金，替代的自由刑的换算比例是什么？
25.	根据《监狱法》第15条，被判处有期徒刑的罪犯，若在交付执行刑罚前，剩余刑期在多长时间以下的，由看守所代为执行？
26.	联系宪法中关于基本制度和基本人民权利义务的规定，论述宪法是如何保障中国式现代化的？
27.	联系习近平法治思想，论述依法治国和依宪治国。
基于刚才上下部分两个视频，请你开始针对问题列表的每个问题进行作答。对于无法推出答案的问题，请回答我暂时从视频中找不到答案。
"""
response = chat.send_message(prompt)
# 打印响应，渲染任何 Markdown
print(response.text)

ERROR:tornado.access:500 POST /v1beta/models/gemini-exp-1206:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 163965.47ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-exp-1206:generateContent?%24alt=json%3Benum-encoding%3Dint: TypeError: Failed to fetch

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")